# Know Error
🟩 The bot tends to genrate multiple responese for a single reply (fixed)

🟩 Genrates multiple responses while working with media files (fixed)

🟩 Is able to remember images only time it is uploaded.(fixed)

🟥 Improving the video responses

🟨 Finding a way to keep the converstion for the videos conversation


# Goal
1. Fix the 2000 char limit. ✅
2. Saving the chats. ✅
3. Only reply when tagged or replied ✅
4. Consinent chats ✅
5. Image input ✅
6. Mutimedia input ✅
7. Webhook

In [ ]:
#@title <b>v-- If you play on mobile, tap this to open music player and play the white noise to keep tab running in the background. or google will kill your api
%%html
<b>Press play on the music player to keep the tab alive, then start block below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

# Install requirments

In [ ]:
!pip install -q -U google-generativeai
!pip install Discord
!pip install textract
!pip install pydub
!pip install sox

# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

text_generation_config = {
    "temperature": 0.9,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safetyy_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

system_instruction = "Remember to respond in the format!! The message is described in the format: User {user_id}: {User message}Bot: {Bot name}: {Bot response} I will give you an example. User{user_id}: Hey can you tell me the first 10 numbers. Bot:{Bot name}: Yes, I can do that. The first 10 numbers are: 1,2,3,4,5,6,7,8,9,10. **NOW PAY ATTENTION** Now if the user input is: User:(user_id): What is the sum of it? Now instead of asking user about the numbers you should refer back to the conversation. Bot: {bot name}: The answer is 55. \\" # @param {type:"string"}

text_model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest", generation_config=text_generation_config, system_instruction=system_instruction, safety_settings=safetyy_settings)

**For video**

In [14]:
# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "/content/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps
  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}:{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)
  def set_file_response(self, response):
    self.response = response

In [3]:
# @title Functions

from random import choice, randint
import PIL.Image
import os
from PIL import Image
import requests
from urllib.parse import urlparse, unquote
from pydub import AudioSegment
import textract
import re
import cv2
import shutil


def extract_text(file_path: str) -> str:
    """
    Extracts text from a document
    """
    # Extract text from the document
    text = textract.process(file_path).decode('utf-8')

    return text

def get_extension_from_link(attachment_link: str) -> str:
    """
    Extracts the file extension from the attachment link.
    """
    parsed_link = urlparse(unquote(attachment_link))
    path = parsed_link.path
    extension = path.split('.')[-1]
    return extension.split('?')[0]  # This will remove any query parameters


def download_file(attachment_link: str, user_id: str) -> str:
    """
    Downloads the file and returns the local file path with the correct extension,
    ensuring that the filename is organized without URL parameters.
    """
    extension = get_extension_from_link(attachment_link)
    filename = f'file_{user_id}.{extension}'
    response = requests.get(attachment_link)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded successfully: {filename}")
    else:
        print(f"Failed to download file. HTTP response code: {response.status_code}")
    return filename


def get_chat_history(user_id: str, custom_path: str = '') -> str:
    sanitized_user_id = ''.join(char for char in user_id if char.isalnum() or char in ('-', '_'))
    file_name = f'{sanitized_user_id}_chat_log.txt'
    if custom_path:
        full_path = os.path.join(custom_path, file_name)
    else:
        full_path = file_name

    chat_history = 'The chat history is empty'
    if os.path.exists(full_path):
        with open(full_path, 'r') as file:
            chat_history = file.read()
    return chat_history

def log_to_file(user_id: str, user_input: str, bot_response: str, custom_path: str = ''):
    # Sanitize the user_id to remove any characters that are not allowed in file names
    sanitized_user_id = ''.join(char for char in user_id if char.isalnum() or char in ('-', '_'))
    file_name = f'{sanitized_user_id}_chat_log.txt'

    # Use the custom path if provided
    full_path = os.path.join(custom_path, file_name) if custom_path else file_name

    with open(full_path, 'a') as file:
        file.write(f'User ({user_id}): {user_input}\n')
        file.write(f'Bot: {bot_response}\n\n')

def log_to_file_doc(user_id: str, user_input: str, bot_response: str, string_doc: str, custom_path: str = ''):
    # Sanitize the user_id to remove any characters that are not allowed in file names
    sanitized_user_id = ''.join(char for char in user_id if char.isalnum() or char in ('-', '_'))
    file_name = f'{sanitized_user_id}_chat_log.txt'

    # Use the custom path if provided
    full_path = os.path.join(custom_path, file_name) if custom_path else file_name

    with open(full_path, 'a') as file:
        file.write(f'User ({user_id}): {user_input}\n')
        file.write(f'Document: {string_doc}\n')
        file.write(f'Bot: {bot_response}\n\n')


def get_response(user_input: str, user_id: str, custom_path: str = '/content/drive/MyDrive/Discord_bot/') -> str:
    chat_history = get_chat_history(user_id, custom_path)
    # Combine the chat history with the current input
    input_tagger= f'User ({user_id}): {user_input}\n'
    combined_input = chat_history + f' The current input by the user is: {input_tagger}'
    # Here we use the Gemini API to get a response, providing the combined_input
    response = text_model.generate_content(combined_input)
    print(response.text)
    return response.text

def get_response_img(user_input: str, user_id: str, attachment_link: str, custom_path: str = '/content/drive/MyDrive/Discord_bot/') -> tuple:
    # Download the image with the correct extension
    image_filename = download_file(attachment_link, user_id)

    # Rest of your existing code
    chat_history = get_chat_history(user_id, custom_path)
    input_tagger= f'User ({user_id}): {user_input}\n'
    combined_input = chat_history + f' The current input by the user is: {input_tagger}' + "The user has uploaded an image:"

    # Open the image using the dynamic filename
    img = Image.open(image_filename)

    # Using Gemini API to get, by combining the input and img into a single list
    response = text_model.generate_content([combined_input, img])
    print(response.text)
    return image_filename, response.text


def get_response_doc(user_input: str, user_id: str, attachment_link: str, custom_path: str = '/content/drive/MyDrive/Discord_bot/') -> tuple:
    doc_filename = download_file(attachment_link, user_id)
    string_in_file = "([{The user has uploaded a text file \n The text file starts below(The end of the text file will be indicated by a similar text to this but it will have the number '5234791368' at the end): }])" + extract_text(doc_filename) + "([{The user uploaded document has ended. 5234791368}])"
    chat_history = get_chat_history(user_id, custom_path)
    input_tagger = f'User ({user_id}): {user_input}\n'
    combined_input = chat_history + f' The current input by the user is: {input_tagger}' + string_in_file
    response = text_model.generate_content(combined_input)
    print(response.text)
    return string_in_file, response.text

def get_response_aud(user_input: str, user_id: str, attachment_link: str, custom_path: str = '/content/drive/MyDrive/Discord_bot/') -> tuple:
    audio_filename = download_file(attachment_link, user_id)
    #uploading the audio_filename to the files api
    your_file = genai.upload_file(path = f'/content/{audio_filename}')
    chat_history = get_chat_history(user_id, custom_path)
    input_tagger= f'User ({user_id}): {user_input}\n'
    combined_input = chat_history + f' The current input by the user is: {input_tagger}' + "The user has uploaded an audio file. Listen carefully to the following audio file and answer any questioned asked, or if no questione is asked then give a brief summery:"
    # Using Gemini API to get, by combining the input and aud into a single list
    response = text_model.generate_content([combined_input, your_file])
    print(response.text)
    return your_file, response.text

#for giving the ability to have chat history while a image is involved in the converstion
def get_response_img_countinue(user_input: str, user_id: str, img_path: str, custom_path: str = '/content/drive/MyDrive/Discord_bot/') -> str:
    chat_history = get_chat_history(user_id, custom_path)
    # Combine the chat history with the current input
    input_tagger= f'User ({user_id}): {user_input}\n'
    combined_input = chat_history + f' The current input by the user is: {input_tagger} + The user has uploaded a photo in the past, responed with the content of the image if the user ask or it is mentioned in the flow of conversation: '
    # Here we use the Gemini API to get a response, providing the combined_input
    img = Image.open(img_path)
    response = text_model.generate_content([combined_input, img])
    print(response.text)
    return response.text

#for giving the ability to have chat history while a audio is involved in the converstion
def get_response_aud_countinue(user_input: str, user_id: str, your_file, custom_path: str = '/content/drive/MyDrive/Discord_bot/') -> str:
    chat_history = get_chat_history(user_id, custom_path)
    #basic_promot = "" # @param {type:"string"}
    input_tagger= f'User ({user_id}): {user_input}\n'
    combined_input = chat_history + f' The current input by the user is: {input_tagger} + The user has uploaded a audio clip in the past, responed with the content of the audio clip if the user ask or it is mentioned in the flow of conversation:  '
    response = text_model.generate_content([combined_input, your_file])
    print(response.text)
    return response.text

#currently converstion history is not saved when files involves video, will try to change it in future
def get_response_vid(user_input: str, user_id: str, attachment_link: str, custom_path: str = '/content/drive/MyDrive/Discord_bot/') -> str:
    #chat_history = get_chat_history(user_id, custom_path)
    input_tagger= f'User ({user_id}): {user_input}\n'
    video_file_name = download_file(attachment_link, user_id)
    #combined_input = chat_history + f' The current input by the user is: {input_tagger} + The user has uploaded a video: '
    extract_frame_from_video(video_file_name)
    files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
    files = sorted(files)
    files_to_upload = []
    for file in files:
      files_to_upload.append(
          File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
# Only upload a 10 second slice of files to reduce upload time or change to true value to upload the full lenght
    full_video = True

    uploaded_files = []
    print(f'Uploading {len(files_to_upload) if full_video else 10} files. This might take a bit...')

    for file in files_to_upload if full_video else files_to_upload[40:50]:
      print(f'Uploading: {file.file_path}...')
      response = genai.upload_file(path=file.file_path)
      file.set_file_response(response)
      uploaded_files.append(file)
      def make_request(prompt, files):
          request = [prompt]
          for file in files:
            request.append(file.timestamp)
            request.append(file.response)
          return request

# Using the request functions
    request = make_request(input_tagger, uploaded_files)
    response = text_model.generate_content(request,
                                  request_options={"timeout": 600})
    print(response.text)
    return(response.text)

# Running The Bot

In [ ]:
from typing import Final
import os
from discord import Intents, Client, Message
from google.colab import userdata
import PIL.Image

# STEP 0: LOAD OUR TOKEN FROM SOMEWHERE SAFE
TOKEN: Final[str] = userdata.get('DISCORD_TOKEN')
#Using them for checking if the user has added image or audio to the converstion
imgtagger = 0
photo = None
audtagger = 0

# STEP 1: BOT SETUP
intents: Intents = Intents.default()
intents.message_content = True
client: Client = Client(intents=intents)

# STEP 2: MESSAGE FUNCTIONALITY
async def send_message(message: Message, response: str, user_id: str) -> None:

    # Split the response into chunks of 2000 characters to overcome discord limitions of 2000 characters
    chunks = [response[i:i+2000] for i in range(0, len(response), 2000)]
    for chunk in chunks:
        # Send each chunk as a separate message to the channel
        await message.channel.send(chunk)

# STEP 3: HANDLING THE STARTUP FOR OUR BOT
@client.event
async def on_message(message: Message) -> None:

    global imgtagger, photo, file_path, audtagger, gemini_aud_link

    # Ignore messages sent by the bot itself
    if message.author == client.user:
        return

    #A command to remove any attachments to the covnerstion with the user
    if message.content.startswith('!reset media'):
        imgtagger = 0
        audtagger = 0
        await message.channel.send('Media counters reset.')
        user_id: str = str(message.author.id)
        sanitized_user_id = ''.join(char for char in user_id if char.isalnum() or char in ('-', '_'))
        file_name = f'{sanitized_user_id}_chat_log.txt'

        # Use the custom path if provided
        custom_path: str = '/content/drive/MyDrive/Discord_bot/'
        full_path = os.path.join(custom_path, file_name) if custom_path else file_name

            #For telling the bot that the file is removed and bot is less likely to hallucinate
        with open(full_path, 'a') as file:
            file.write(f'The user has removed all the media. If the user ask for any question realted to any media file. Ask the user to upload the media again.\n')
        return

    # Check if the bot is mentioned or if the message is a reply to the bot
    bot_mentioned = client.user in message.mentions
    is_reply = message.reference is not None and message.reference.resolved.author == client.user

    # Respond only if the bot is mentioned or if it's a reply to the bot's message
    if bot_mentioned or is_reply:
        user_id: str = str(message.author.id)  # Get the user's ID as a string
        username: str = str(message.author)
        channel: str = str(message.channel)

        # Check for attachments in the message
        if message.attachments:
            # Handle the attachments based on their type
            for attachment in message.attachments:
                # Check the file extension
                if any(attachment.filename.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.heic', '.webp', 'heif']):
                    # Call a function to handle image attachments
                    attach_url= attachment.url
                    print(f'[{channel}] {user_id}: "{message.content}"')
                    image, response = get_response_img(message.content, user_id, attach_url)
                    file_path = f'/content/{image}'
                    custom_log_path = '/content/drive/MyDrive/Discord_bot/'
                    log_to_file(user_id, message.content, response, custom_log_path)
                    imgtagger = 1
                    await send_message(message, response, user_id)
                    break
                elif any(attachment.filename.lower().endswith(ext) for ext in ['.pdf', '.doc', '.docx', '.txt', '.xlsx', '.ppt', '.pptx']):
                    # Call a function to handle document attachments
                    attach_url= attachment.url
                    print(f'[{channel}] {user_id}: "{message.content}"')
                    string_in_file, response = get_response_doc(message.content, user_id, attach_url)
                    custom_log_path = '/content/drive/MyDrive/Discord_bot/'
                    log_to_file_doc(user_id, message.content, response, string_in_file, custom_log_path)
                    await send_message(message, response, user_id)
                    break
                elif any(attachment.filename.lower().endswith(ext) for ext in ['.wav', '.mp3', '.aiff', '.aac', '.ogg', '.flac']):
                    # Call a function to handle document attachments
                    attach_url= attachment.url
                    print(f'[{channel}] {user_id}: "{message.content}"')
                    gemini_aud_link, response = get_response_aud(message.content, user_id, attach_url)
                    custom_log_path = '/content/drive/MyDrive/Discord_bot/'
                    log_to_file(user_id, message.content, response, custom_log_path)
                    await send_message(message, response, user_id)
                    audtagger = 1
                    return audtagger
                elif any(attachment.filename.lower().endswith(ext) for ext in ['.mp4']):
                    response = "Conversation involving video file doesn't support chat history for now!! It might take some time to process a video file, you will be menionted when the response is ready! "
                    await send_message(message, response, user_id)
                    attach_url= attachment.url
                    print(f'[{channel}] {user_id}: "{message.content}"')
                    respons = get_response_vid(message.content, user_id, attach_url)
                    response = f"<@{user_id}> {respons}"
                    await send_message(message, response, user_id)
                    break
                else:
                    # Handle other types of attachments or unknown file extensions
                    response = f"You've sent an attachment, but I'm not sure what it is. **Note: .mp4, .mov and other video format are currently not supported.**\nAttachment URL: {attachment.url}"
                await send_message(message, response, user_id)
                break

        elif imgtagger == 0 and audtagger == 0:
            # No attachments, proceed with handling text messages
            print(f'[{channel}] {user_id}: "{message.content}"')
            response = get_response(message.content, user_id)
            # Define the custom path where you want to save the log
            custom_log_path = '/content/drive/MyDrive/Discord_bot/'
            log_to_file(user_id, message.content, response, custom_log_path)  # Log the conversation with the user's ID
            await send_message(message, response, user_id)

        elif imgtagger == 1:
            print(f'[{channel}] {user_id}: "{message.content}"')
            response = get_response_img_countinue(message.content, user_id, file_path)
            custom_log_path = '/content/drive/MyDrive/Discord_bot/'
            log_to_file(user_id, message.content, response, custom_log_path)  # Log the conversation with the user's ID
            await send_message(message, response, user_id)


        elif audtagger == 1:
            print(f'[{channel}] {user_id}: "{message.content}"')
            response = get_response_aud_countinue(message.content, user_id, gemini_aud_link)
            custom_log_path = '/content/drive/MyDrive/Discord_bot/'
            log_to_file(user_id, message.content, response, custom_log_path)  # Log the conversation with the user's ID
            await send_message(message, response, user_id)

# STEP 5: MAIN ENTRY POINT
async def main() -> None:
    await client.start(TOKEN)

# To run in a cloud service like google colob, kaggle etc
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    !pip install nest_asyncio
    import nest_asyncio
    nest_asyncio.apply()

# Run the main function
await main()